In [1]:
from dotenv import load_dotenv
import os
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
load_dotenv()

c:\Users\Shohruz Ernazarov\miniconda3\envs\rmp-rag\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [2]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [3]:
import json
data = json.load(open("mockData.json"))
data["reviews"]

[{'professor': 'Dr. Jane Smith',
  'star_rating': 4,
  'subject': 'Computer Science',
  'reviews': ['Great professor with clear explanations.',
   'Assignments are challenging but fair.',
   'Very approachable and always willing to help.',
   'The lectures can be a bit fast-paced, but overall, a good learning experience.']},
 {'professor': 'Dr. Michael Johnson',
  'star_rating': 5,
  'subject': 'Mathematics',
  'reviews': ["Best math professor I've ever had!",
   'Makes complex concepts easy to understand.',
   'Always available during office hours.',
   'Engaging lectures that keep you interested.',
   'Highly recommend for anyone interested in math.']},
 {'professor': 'Dr. Emily Chen',
  'star_rating': 3,
  'subject': 'Physics',
  'reviews': ['Lectures are informative but sometimes hard to follow.',
   'The grading is strict, but she is fair.',
   'Labs are interesting, but the workload is heavy.',
   'Good professor, but you need to put in a lot of effort.']},
 {'professor': 'Dr. Ro

In [4]:
processed_data = []
client = OpenAI()

for review in data["reviews"]:
    response = client.embeddings.create(
        input=review["reviews"],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "reviews": review["reviews"],
            "star_rating": review["star_rating"],
            "subject": review["subject"]
        }
    })

In [5]:
processed_data[0]

{'values': [-0.011611851,
  -0.0057255663,
  -0.021924566,
  0.008799291,
  -0.019527195,
  -0.0083707115,
  -0.03214353,
  0.023103163,
  -0.025420176,
  -0.0067066136,
  0.03640255,
  -0.008035882,
  -0.025755005,
  -0.010306019,
  -0.0032679257,
  -0.006957735,
  0.017062858,
  0.047545638,
  0.008270263,
  0.06155486,
  0.027147891,
  -0.03380428,
  0.016875355,
  0.05689405,
  -0.034206074,
  -0.047706358,
  0.020263817,
  0.0010337828,
  -0.008089455,
  0.004342725,
  0.074680135,
  -0.007567123,
  0.0151476385,
  -0.059572678,
  -0.061072707,
  0.017330721,
  -0.0034035312,
  0.006318213,
  -0.00015485815,
  0.031152438,
  0.015053887,
  0.0076943575,
  -0.022768334,
  -0.0014464589,
  0.06134057,
  0.029598834,
  -0.0558226,
  -0.012402046,
  0.023960324,
  0.022687975,
  -0.014370837,
  -0.019701306,
  0.06578709,
  0.028955964,
  -0.08051954,
  0.04623311,
  0.011317202,
  0.03380428,
  0.02330406,
  -0.036723983,
  0.06262631,
  -0.007567123,
  0.03784901,
  0.006130709,
  0

In [6]:
index = pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [7]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}